In [1]:
import os
import re
import comtypes.client as cc
import comtypes
tlb_id = comtypes.GUID("{310BB2DC-AB12-4DE3-839C-1E7CDEAD68A3}")
cc.GetModule((tlb_id, 1, 3))
import comtypes.gen.ct_LocateXT_API as LXT_API

In [48]:
text_file = r"C:\Users\jame9353\Box Sync\Projects\DCGS-A Message Traffic\data\source_data"

In [2]:
find_dates = True #use True or False to search for dates

find_polar_UTM_MGRS = False #use True or False to search for polar coordinates

#gaz_file_path = r''
gaz_file_path = r''
fuzzy_percent_error_level = 0 #0 - 30

#ca_file_path = r''
ca_file_path = r''

In [ ]:
def worker(file):
    ICoordSearch = cc.CreateObject(LXT_API.CoordinateSearching,None,None,LXT_API.ICoordinateSearching2)
    ICoordSearch.UTM_NorthPolar = find_polar_UTM_MGRS
    ICoordSearch.UTM_SouthPolar = find_polar_UTM_MGRS
    ICoordSearch.MGRS_NorthPolar = find_polar_UTM_MGRS
    ICoordSearch.MGRS_SouthPolar = find_polar_UTM_MGRS
  
    ILxtMgr = cc.CreateObject(LXT_API.LocateXT_Manager2, None, None, LXT_API.ILocateXT_Manager3)
    ILxtMgr.SetSearcher(ICoordSearch)
  

    if find_dates:
        IDateSearch = cc.CreateObject(LXT_API.DateSearching,None,None,LXT_API.IDateSearching)
        ILxtMgr.SetSearcher(IDateSearch)

    if gaz_file_path:
        #print("gaz: ",gaz_file_path," err: ",fuzzy_percent_error_level)
        IGazSearch = cc.CreateObject(LXT_API.GazetteerSearching,None,None,LXT_API.IGazetteerSearching)
        IGazSearch.GazetteerFile = os.path.abspath(gaz_file_path)
        IGazSearch.FuzzyErrorLevelPercent= fuzzy_percent_error_level
        ILxtMgr.SetSearcher(IGazSearch)

    if ca_file_path:
        #print("ca: ",ca_file_path)
        ICASearch = cc.CreateObject(LXT_API.CustomAttributeSearching,None,None,LXT_API.ICustomAttributeSearching)
        ICASearch.CustomAttributesFile = os.path.abspath(ca_file_path)
        ILxtMgr.SetSearcher(ICASearch)

    out_path = os.path.abspath (path_to_output)

    #loop until the file can be opened exclusively
    #this ensures the file is done being written/copied
    #and can be processed further
    success = False
    count = 0
    while not success and count < 20:
        try:
            hFile = win32file.CreateFile (
                      full_filename,
                      win32con.GENERIC_READ | win32con.GENERIC_WRITE,
                      0,#exclusive access
                      None,
                      win32con.OPEN_EXISTING,
                      win32con.FILE_ATTRIBUTE_NORMAL,
                      None
                    )

        except:
             #print("Unexpected error:", sys.exc_info()[1])
             #raise
            pass
        else:
            #print("handle = ", hFile)
            win32api.CloseHandle(hFile)
            success = True

        count += 1
        time.sleep(0.5)

        if success:
            #file can be processed
            print("Processing: ",file)
            csvOut = ILxtMgr.Scan(LXT_API.lxtInputTypeFilename, file, LXT_API.lxtOutputTypeCSV)
            csvOutAscii = re.sub(r'[^\x00-\x7f]',r' ',csvOut)#replace every character NOT in 0x00 - 0x7f with space
            #print(csvOutAscii)
            #eliminate embedded carriage returns
            csvByteArray = bytearray(csvOutAscii,'ascii','replace')#also replaces every char NOT ASCII, but with '?'
            insideQuotes = False
            i = 0
            while i < (len(csvByteArray) - 1):
            #if i < 100:
            #  print("processing i = ",i,"  : ",csvByteArray[i],"\n")
                if insideQuotes:
                  #print("insideQuotes\n")
                    if 34 == csvByteArray[i]:
                        if 34 != csvByteArray[i+1]:
                            insideQuotes = False
                        else:
                              i += 1
                    elif (13 == csvByteArray[i]) or (10 == csvByteArray[i]):
                        csvByteArray[i] = 32
                elif 34 == csvByteArray[i]:
                    insideQuotes = True
                #while loop last
                i += 1
      
            #write file
            #print(csvByteArray)      
            fd, tmpName = tempfile.mkstemp(".csv",re.sub(r'[\/:*?"<>|]',r'_',str(datetime.datetime.now())),out_path,True)
            os.write(fd,csvByteArray)
            os.close(fd)
            print("Output: ",tmpName)
             

    
    print("exiting thread")

In [103]:
def extract_tacelint(text, intermediate_file):

    
    arcpy.conversion.ExtractLocationsText(coords, 
                                      intermediate_file, 
                                      None, "FIND_DD_LATLON", "FIND_DD_XYDEG", "FIND_DD_XYPLAIN", "FIND_DM_LATLON", 
                                      "FIND_DM_XYMIN", "FIND_DMS_LATLON", "FIND_DMS_XYSEC", "FIND_DMS_XYSEP", 
                                      "FIND_UTM_MAINWORLD", "DONT_FIND_UTM_NORTHPOLAR", "DONT_FIND_UTM_SOUTHPOLAR", 
                                      "FIND_MGRS_MAINWORLD", "DONT_FIND_MGRS_NORTHPOLAR", "DONT_FIND_MGRS_SOUTHPOLAR", 
                                      "USE_DOT_DECIMAL_MARK", "PREFER_LATLON", "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]];-400 -400 11258999068426.2;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision", 
                                      None, "DONT_USE_FUZZY", None, None, "FIND_DATE_MONTHNAME", "FIND_DATE_M_D_Y", "FIND_DATE_YYYYMMDD", 
                                      "FIND_DATE_YYMMDD", "FIND_DATE_YYJJJ", None, None, None, None, 
                                      None, None, None, 254, 254, "STD_COORD_FMT_DD")
    
    if arcpy.Exists(intermediate_file) == True:
        result = arcpy.GetCount_management(intermediate_file)
        if int(result[0]) >0:
            fields = [f.name for f in arcpy.ListFields(intermediate_file)]
            with arcpy.da.SearchCursor(intermediate_file, fields) as cursor:
                for row in cursor:
                    shape = row[0]
                
                    return shape
        else:
            return "No valid coordinates extracted", 0, 0
    else:
        return "No valid coordinates extracted", 0, 0
    
    del intermediate_file

In [119]:
# Function to watch a folder and detect new images on a 1 second refresh interval
#before = dict ([(f, None) for f in os.listdir (text_file)])
before = {}
count = 0
errors = 0

while True:
    
    
    # Compares the folder contents after the sleep to what existed beforehand, and makes a list of adds and removes
    after = dict ([(f, None) for f in os.listdir (text_file)])
    added = [f for f in after if not f in before]
    removed = [f for f in before if not f in after]

    if added: print("Added: ", ", ".join (added))
    if removed: print("Removed: ", ", ".join (removed))
    before = after

    for file in added:
        if file.endswith(".txt"):
            count +=1
            with open(os.path.join(text_file, file), mode='r') as file:
                tacelint = file.read()
                lines = tacelint.split("\n")
                if lines[1].startswith("MSGID/"):
                    msg_type = lines[1].split("/")[1]
                    if msg_type == 'TACELINT'or msg_type == 'RECCEXREP':
                        for line in lines:
                            int_file = "in_memory/tacelint_file"
                            split_line = line.split("/")
                            if line.startswith("SOI/"):
                                date_obs = split_line[2]
                                date_rep = split_line[3]
                                asset = split_line[5]
                                reference = split_line[7]
                                print(date_obs, date_rep, asset, reference)
                            elif line.startswith("EMLOC/"):
                                coords = split_line[3].split(":")[1]
    
                                if len(split_line) == 7:
                                    semiminor = split_line[4]
                                    semimajor = split_line[4]
                                if len(split_line) == 10:
                                    semiminor = split_line[7]
                                    semimajor = split_line[6]
                                
                                point = extract_tacelint(coords, int_file)
                                if type(point) == tuple:
                                    print(point, semiminor, semimajor)
                                else:
                                    print(point)
                                #except:
                                    #print("Error processing data, passing...")
                            elif line.startswith('EXER/') or line.startswith('OPER/'):
                                op_type = split_line[0]
                                op_name = split_line[1]
                                print(op_type, op_name)
                            
                            elif line.startswith('/'):
                                if len(split_line) == 8:
                                    if msg_type == 'TACELINT' or msg_type == 'RECCEXREP':
                                        classification = split_line[5]
                                        orig_country = split_line[4]
                                        msg_dt = split_line[1]
                                        print(classification, orig_country, msg_dt)
                            elif line.startswith('MISSNID/'):
                                unit = split_line[2]
                                unit_code = split_line[5]
                                print(unit, unit_code)
                        
                            if arcpy.Exists(int_file) == True:
                                arcpy.Delete_management(int_file)
                                
        print("Processed " + str(count) + " files...")
                                
    if count == 10:
        print("Exiting")
        break

Added:  example1.txt, example2.txt, example3.txt, example4.txt, recexrep.txt, tacrep1.txt, tacrep2.txt, tacrep3.txt, tacrep4.txt, tacrep5.txt
EXER BRAVE SHIELD 93
UNCLASSIFIED USA 20090214T132530Z
060812Z 060821Z HIGHBLOW 3R456
(-79.02380555555555, 35.17369444444438) 35NM 35NM
060942Z 060821Z ROUNDFACE 3R456
(-94.01169396025898, 38.95563519524666) 25KM 100KM
Processed 1 files...
Processed 2 files...
OPER WET BIRD
UNCLASSIFIED USA 20090214T132530Z
311015Z 312020Z SOFTBALL -
(-1.404583333333278, 52.22924999999998) 12.1NM 25.6NM
Processed 3 files...
EXER BRAVE SHIELD 93
UNCLASSIFIED USA 20090214T132530Z
060812Z 060821Z HIGHBLOW 3R456
(-79.02380555555555, 35.17369444444438) 35NM 35NM
060942Z 060821Z ROUNDFACE 3R456
('No valid coordinates extracted', 0, 0) 25KM 100KM
Processed 4 files...
EXER BLACK PIG
UNCLASSIFIED USA 20090214T132530Z
I GE CORPS Z198A
Processed 5 files...
Processed 6 files...
Processed 7 files...
Processed 8 files...
Processed 9 files...
Processed 10 files...
Exiting


In [60]:
arcpy.Delete_management("in_memory/tacelint_file")

<Result 'true'>

In [61]:
arcpy.Exists("in_memory/tacelint_file")

False

Unable to create process using 'C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Scripts\pip-script.py" install comtypes'


In [1]:
import sys
!{sys.executable} -m pip install comtypes

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
